## Clause Detection Using Pre-Trained Model

In [18]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import utils as du
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


In [19]:
model_path = "../models/CC_BERT/CC_model_detect" 
device = torch.device("cpu")

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50500, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
           

In [20]:
# NOTE: If this throws an error, it is because you may have uniquely constructed your test set from training_model.ipynb. 
# Because of randonmess, different contracts may be selected for training and testing. 
# Just go into the data/test_contracts directory and pull the name of a single contract to fix this.
single_contract = "../data/spend_network/fresh_contracts_identified_txt/241209_con_7063_desnz_erm_nzm_modelling_contract_award_form_and_schedules_v1_redacted.txt"
contract_folder = "../data/test_contracts"

In [21]:
#you can input contract_dir or single_contract as the argument to this function
processed_contracts = du.load_unlabelled_contract(contract_folder)

In [22]:
processed_contracts

,index,text
0,000038611.txt,Exhibit 10.14 SEPARATION AGREEMENT between Ray...
1,000038611.txt,"1.2 This termination shall extend to all, if a..."
2,000038611.txt,2. RETIREMENT FROM ALL FORMS OF OFFICE 2.1 Mr ...
3,000038611.txt,2.2 Mr Burel likewise shall retire from his of...
4,000038611.txt,2.3 Mr Burel shall resign from his office in s...
...,...,...
376604,000008913.txt,38 PROTECTION OF THE TRUSTEE Any sale by the T...
376605,000008913.txt,39 APPLICATION FOR LISTING OR ADMISSION TO TRA...
376606,000008913.txt,"shall, at its expense, make application for, a..."
376607,000008913.txt,40 RELATIONSHIP OF PLAN TO CONTRACT OF EMPLOYM...


In [23]:
texts = processed_contracts["text"].tolist()

In [ ]:
results, probs = du.predict_climatebert(texts, tokenizer, device, model)

In [ ]:
result_df, result_df_true = du.create_result_df(results, processed_contracts)

In [ ]:
result_df

,prediction,sentence,contains_climate_keyword
0,0,Copyright(c)2024 The Chancery Lane Project Per...,False
1,0,"THE SOFTWARE IS PROVIDED ""AS IS"", WITHOUT WARR...",False
2,0,IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOL...,False


In [ ]:
probs

array([[9.9996793e-01, 3.2072399e-05],
       [9.9996758e-01, 3.2387499e-05],
       [9.9995756e-01, 4.2390315e-05]], dtype=float32)

In [ ]:
#note: This function is designed to work for single contracts and will create a single file for multiple contracts
highlighted_output = du.highlight_climate_content(result_df)

In [ ]:
#save highlighted output to a file; commented so it doesn't save every time this script runs 
du.save_file("highlighted_output.html", highlighted_output)

## Using the Detector

In [ ]:
contract_df = du.create_contract_df(
    result_df, processed_contracts, labelled=False
)

In [ ]:
cat0, cat1, cat2, cat3 = du.create_threshold_buckets(contract_df)

In [ ]:
du.print_percentages(cat0, cat1, cat2, cat3, contract_df)

'unlikely'

I split the data 60% including a clause and 40% not including a clause so this looks good! 

### Testing for a Single File

In [ ]:
# note: this doesn't work if you are working with a contract database; it is designed for a single file

# If you want to see this functionality, run the code using the single file path
du.print_single(cat0, cat1, cat2, cat3)

unlikely
